# Models, Prompts and Parsers

In [1]:
import os
import openai

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai.api_key = os.environ["OPENAI_API_KEY"]

## Chat API: OpenAI

In [2]:
from openai import OpenAI

def get_completion(prompt, model="gpt-4o-mini"):
    messages = [{"role": "user", "content": prompt}]
    
    client = OpenAI(api_key=openai.api_key)
    response = client.chat.completions.create(model=model, messages=messages, temperature=0)

    return response.choices[0].message.content

In [3]:
get_completion("What's 1+1?")

'1 + 1 equals 2.'

## Chat API: LangChain

In [4]:
from langchain.prompts import ChatPromptTemplate

template = """Translate the text \
that is delimited by triple backticks \
into a style that is {style}. \
text: ```{text}```
"""

prompt = ChatPromptTemplate.from_template(template)
prompt

ChatPromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['style', 'text'], input_types={}, partial_variables={}, template='Translate the text that is delimited by triple backticks into a style that is {style}. text: ```{text}```\n'), additional_kwargs={})])

In [5]:
# Customer request
customer_style = """American English \
in a calm and respectful tone
"""

customer_text = """
Arrr, I be fuming that me blender lid \
flew off and splattered me kitchen walls \
with smoothie! And to make matters worse, \
the warranty don't cover the cost of \
cleaning up me kitchen. I need yer help \
right now, matey!
"""

In [6]:
from pprint import pprint

customer_prompt = prompt.format_messages(style=customer_style, text=customer_text)[0].content
pprint(customer_prompt)

('Translate the text that is delimited by triple backticks into a style that '
 'is American English in a calm and respectful tone\n'
 '. text: ```\n'
 'Arrr, I be fuming that me blender lid flew off and splattered me kitchen '
 "walls with smoothie! And to make matters worse, the warranty don't cover the "
 'cost of cleaning up me kitchen. I need yer help right now, matey!\n'
 '```\n')


In [7]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(temperature=0)
pprint(model.invoke(customer_prompt).content)

("I'm really frustrated that my blender lid flew off and splattered my kitchen "
 "walls with smoothie! And to make matters worse, the warranty doesn't cover "
 'the cost of cleaning up my kitchen. I could really use your help right now, '
 'friend.')


In [8]:
# Service response
service_style = """\
a polite tone \
that speaks in English Pirate\
"""

service_text = """Hey there customer, \
the warranty does not cover \
cleaning expenses for your kitchen \
because it's your fault that \
you misused your blender \
by forgetting to put the lid on before \
starting the blender. \
Tough luck! See ya!
"""

In [9]:
service_prompt = prompt.format_messages(style=service_style, text=service_text)[0].content
pprint(model.invoke(service_prompt).content)

("Ahoy there, valued customer! Regrettably, the warranty be not coverin' the "
 "cost o' cleanin' yer galley due to yer own negligence. Ye see, 'twas yer own "
 "doin' that ye forgot to secure the lid afore startin' the blender. 'Tis a "
 'tough break, matey! Fare thee well!')


## Output parsers

In [10]:
customer_review = """\
This leaf blower is pretty amazing. It has four settings:\
candle blower, gentle breeze, windy city, and tornado. \
It arrived in two days, just in time for my wife's \
anniversary present. \
I think my wife liked it so much she was speechless. \
So far I've been the only one using it, and I've been \
using it every other morning to clear the leaves on our lawn. \
It's slightly more expensive than the other leaf blowers \
out there, but I think it's worth it for the extra features.
"""

review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product \
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

Format the output as JSON with the following keys:
gift
delivery_days
price_value

text: {text}
"""

In [11]:
prompt = ChatPromptTemplate.from_template(review_template)
messages = prompt.format_messages(text=customer_review)
response = model.invoke(messages).content
print(response)
type(response)  # str -> we can't access the dict keys

{
    "gift": true,
    "delivery_days": 2,
    "price_value": "It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."
}


str

In [12]:
# Describe the output structure
from langchain.output_parsers import ResponseSchema, StructuredOutputParser

gift_schema = ResponseSchema(
    name="gift",
    description="Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.",
)
delivery_days_schema = ResponseSchema(
    name="delivery_days",
    description="How many days did it take for the product to arrive? If this information is not found, output -1.",
)
price_value_schema = ResponseSchema(
    name="price_value",
    description="Extract any sentences about the value or price, and output them as a comma separated Python list.",
)

response_schemas = [gift_schema, delivery_days_schema, price_value_schema]

output_parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = output_parser.get_format_instructions()
print(format_instructions)  # this generated instructions will be added to the prompt

The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```":

```json
{
	"gift": string  // Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.
	"delivery_days": string  // How many days did it take for the product to arrive? If this information is not found, output -1.
	"price_value": string  // Extract any sentences about the value or price, and output them as a comma separated Python list.
}
```


In [13]:
review_template = """\
For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? \
Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the product\
to arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,\
and output them as a comma separated Python list.

text: {text}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template=review_template)
messages = prompt.format_messages(text=customer_review, format_instructions=format_instructions)
print(messages[0].content)

For the following text, extract the following information:

gift: Was the item purchased as a gift for someone else? Answer True if yes, False if not or unknown.

delivery_days: How many days did it take for the productto arrive? If this information is not found, output -1.

price_value: Extract any sentences about the value or price,and output them as a comma separated Python list.

text: This leaf blower is pretty amazing. It has four settings:candle blower, gentle breeze, windy city, and tornado. It arrived in two days, just in time for my wife's anniversary present. I think my wife liked it so much she was speechless. So far I've been the only one using it, and I've been using it every other morning to clear the leaves on our lawn. It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features.


The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "```json" and "```"

In [14]:
response = model.invoke(messages)
print(response.content)

```json
{
	"gift": false,
	"delivery_days": 2,
	"price_value": ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]
}
```


In [15]:
output_dict = output_parser.parse(response.content)
print(output_dict)
type(output_dict)  # dict

{'gift': False, 'delivery_days': 2, 'price_value': ["It's slightly more expensive than the other leaf blowers out there, but I think it's worth it for the extra features."]}


dict